In [1]:
#Importing the required libraries
import numpy as np
from numpy import random
import scipy.stats as stats
from drawFromADist import drawFromADist
import matplotlib.pyplot as plt
from scipy.linalg import norm
from sklearn.mixture import GaussianMixture

In [2]:
def find(a) :
    '''This function is written to convert the matlab code into python'''
    return np.array([i for i,val in enumerate(a) if val > 0])

In [3]:
def generate_gaussian(mean=[0,1],sigma=[0.2,0.2],plot=False) :
    '''returns a randomly chosen point from a gaussian distribution'''
    if plot :
        x = np.linspace(-1,3,1000)
        pdf = np.zeros(shape=x.shape)
        y=np.array([])
        for m,s in zip(mean,sigma) :
            pdf += stats.norm.pdf(x, m, s)
        plt.plot(x, pdf)
    
    choice=np.random.choice(mean)
    return np.random.normal(mean[choice], sigma[choice])    

Implementation of GaussianMixture is taken from sklearn library and is used to sample more deltas during retrival.

In [4]:
def temporary_context_model(schedule) :
    N_WORLD_FEATURES = 5
    N_ITEMS = 10
    ENCODING_TIME = 500
    TEST_TIME = 20

    #we are going to model the world as a set of N continuous-valued features.
    #we will model observations of states of the world as samples from N
    #Gaussians with time-varying means and fixed variance. For simplicity,
    #assume that agents change nothing in the world.

    #first fix the presentation schedule; I'm assuming its random

    
    schedule_load = ENCODING_TIME/np.median(np.diff(schedule,1))                
    encoding = np.zeros((N_ITEMS,N_WORLD_FEATURES+1))

    world_m = random.randint(1,10, size=(N_WORLD_FEATURES))       #Generating the world randomly       
    world_var = 1
    #delta = 0.05                       #what does this parameter affect?
    beta_param = 0.98               #what does this parameter affect?
    m = 0  #Changed from zero to one as code is in python

    #simulating encoding
    deltas=[]
    for time in range(1,ENCODING_TIME+1) :
        delta=generate_gaussian()
        world_m = beta_param*world_m + delta
        world = random.normal(world_m, world_var)
        #any item I want to encode in memory, I encode in association with the
        #state of the world at that time.
        if m < N_ITEMS :
            if(time==schedule[m]):
                encoding[m,0:5] =  world
                encoding[m,5]=m             #encoding world_context and item
                m =  m + 1      
                deltas.append(delta)
    
    EM = GaussianMixture(n_components=2,covariance_type='spherical')
    deltas=np.array(deltas).reshape(-1, 1)
    EM = EM.fit(deltas)
    
    out=np.zeros(TEST_TIME)
    while(time<ENCODING_TIME+TEST_TIME):
        #the state of the world is the retrieval cue
        delta=EM.sample()[0][0]
        world_m = beta_param*world_m + delta
        world = random.normal(world_m, world_var)
        soa=np.zeros(N_ITEMS)
        for m in range(N_ITEMS):
            t=world
            t=np.append(t,m)
            #print(t)
            soa[m]=np.dot(encoding[m],t.T)  #finding association strengths
            #print(soa)
        soa=soa/norm(soa)             #normalize
        out[time-ENCODING_TIME] = find(drawFromADist(soa))    #success is number of unique retrievals
        time = time + 1
    
    return len(np.unique(out)),schedule_load 

In [5]:
schedule = np.array([1, 100, 199, 298, 397, 496, 497, 498, 499, 500]) #The optimal schedule
out=[]
for i in range(50):
    success,schedule_load=temporary_context_model(schedule)
    out.append(success)
print('Mean Successful retrivals over multiple trials:'+ str(np.mean(out)))
print('Schedule Load: '+str(schedule_load))

Mean Successful retrivals over multiple trials:8.42
Schedule Load: 5.05050505050505
